<a href="https://colab.research.google.com/github/SaamarthMeston/Capstone-Project-2---Rossmann-s-sales-prediction/blob/main/Rossmann_Sales_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Sales Prediction : Predicting sales of a major store chain Rossmann</u></b>

## <b> Problem Description </b>

### Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

### You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

## <b> Data Description </b>

### <b>Rossmann Stores Data.csv </b> - historical data including Sales
### <b>store.csv </b> - supplemental information about the stores


### <b><u>Data fields</u></b>
### Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* #### Id - an Id that represents a (Store, Date) duple within the test set
* #### Store - a unique Id for each store
* #### Sales - the turnover for any given day (this is what you are predicting)
* #### Customers - the number of customers on a given day
* #### Open - an indicator for whether the store was open: 0 = closed, 1 = open
* #### StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* #### SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* #### StoreType - differentiates between 4 different store models: a, b, c, d
* #### Assortment - describes an assortment level: a = basic, b = extra, c = extended
* #### CompetitionDistance - distance in meters to the nearest competitor store
* #### CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* #### Promo - indicates whether a store is running a promo on that day
* #### Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* #### Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* #### PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
from scipy import stats
import sklearn as sl

Mounting The  google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Data Sets and Creating Pandas Data Frames

In [3]:
Path1 = "/content/drive/MyDrive/Capstone Project-2 (Rossmann Retail Sales Prediction) /Data & Resource/Rossmann Stores Data.csv"
Path2 = "/content/drive/MyDrive/Capstone Project-2 (Rossmann Retail Sales Prediction) /Data & Resource/store.csv"

In [4]:
Rossmann_df = pd.read_csv(Path1)
Store_df = pd.read_csv(Path2)

<ipython-input-4-93fabc0ddddc>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  Rossmann_df = pd.read_csv(Path1)


Data Exploration

Rossmann Dataframe

In [5]:
Rossmann_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [6]:
Rossmann_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [7]:
Rossmann_df.describe(include='all')

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
count,1.017209e+06,1.017209e+06,1017209,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1017209,1.017209e+06
unique,NaN,NaN,942,NaN,NaN,NaN,NaN,5,NaN
top,NaN,NaN,2015-07-31,NaN,NaN,NaN,NaN,0,NaN
freq,NaN,NaN,1115,NaN,NaN,NaN,NaN,855087,NaN
mean,5.584297e+02,3.998341e+00,NaN,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,NaN,1.786467e-01
std,3.219087e+02,1.997391e+00,NaN,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,NaN,3.830564e-01
min,1.000000e+00,1.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00
25%,2.800000e+02,2.000000e+00,NaN,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00
50%,5.580000e+02,4.000000e+00,NaN,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00
75%,8.380000e+02,6.000000e+00,NaN,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,NaN,0.000000e+00


The Rossmann DataSet contains 1017209 rows and 9 Features with Store, DayofWeek, Customers, Open, Promo and SchoolHoliday being Intiger type and Date and StateHoliday being Object type

Store Data Frame

In [8]:
Store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [9]:
Store_df.describe(include='all')

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1115.00000,1115,1115,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000,571
unique,NaN,4,3,NaN,NaN,NaN,NaN,NaN,NaN,3
top,NaN,a,a,NaN,NaN,NaN,NaN,NaN,NaN,"Jan,Apr,Jul,Oct"
freq,NaN,602,593,NaN,NaN,NaN,NaN,NaN,NaN,335
mean,558.00000,NaN,NaN,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573,NaN
std,322.01708,NaN,NaN,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935,NaN
min,1.00000,NaN,NaN,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000,NaN
25%,279.50000,NaN,NaN,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000,NaN
50%,558.00000,NaN,NaN,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000,NaN
75%,836.50000,NaN,NaN,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000,NaN


The Store DataSet contains 1115 rows and 10 Features with store and Promo2 being Intiger type, CompetitionDistance, CompetitionOpenSinceMonth,	CompetitionOpenSinceYear being Float Type and StoreType,	Assortment, PromoInterval being Object Type. 


Data Cleaning

looking for Null values in Rossmann Dataset.

In [10]:
Rossmann_df.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

Looking for Null Values in Store Dataset.

In [11]:
Store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [12]:
Store_df.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

There are many Null values in **CompetitionOpenSinceMonth** ,**CompetitionOpenSinceYear**, **Promo2SinceWeek**, **Promo2SinceYear**, **Promo2SinceYear** and 3 Null values in **CompetitionDistance**.

**Data Cleaning**

CompetitionDistance


In [13]:
Store_df[Store_df['CompetitionDistance'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


We will fill NAN values of CompetitionDistance with zero as this means they dont have a competetior in thier vicinity and thus ompetitionOpenSinceMonth	CompetitionOpenSinceYear NAN values related to these rows should alos be Zero.

In [14]:
Store_df['CompetitionDistance'].fillna(value= 0, inplace=True)

In [15]:
Store_df[Store_df['CompetitionDistance'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


In [16]:
Store_df['CompetitionDistance'] = Store_df['CompetitionDistance'].astype(int,copy =  False)

In [18]:
Store_df['CompetitionOpenSinceMonth'][Store_df['CompetitionDistance'] == 0]

290   NaN
621   NaN
878   NaN
Name: CompetitionOpenSinceMonth, dtype: float64

In [28]:
Month_mode = Store_df['CompetitionOpenSinceMonth'].mode()

In [27]:
Year_mode= Store_df['CompetitionOpenSinceYear'].mode()

In [29]:
Store_df['CompetitionOpenSinceMonth'].fillna(value= Month_mode, inplace=True)

In [30]:
Store_df['CompetitionOpenSinceYear'].fillna(value= Year_mode, inplace=True)

In [33]:
Store_df.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            0
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [24]:
Store_df['CompetitionOpenSinceMonth'][Store_df['CompetitionDistance'] == 0].replace(NaN, 0)

NameError: ignored

In [26]:
Store_df['CompetitionOpenSinceMonth'] = Store_df['CompetitionOpenSinceMonth'].astype(int,copy =  False)

IntCastingNaNError: ignored

In [24]:
Store_df.loc(Store_df['CompetitionDistance']=='0')

TypeError: ignored

**CompetitionOpenSinceMonth**

In [ ]:
Store_df['CompetitionOpenSinceMonth'].fillna(value=0, inplace=True)

**CompetitionOpenSinceYear**

In [ ]:
Store_df['CompetitionOpenSinceYear'].fillna(value=0, inplace=True)

**Promo2SinceWeek**

In [ ]:
Store_df['Promo2SinceWeek'].fillna(value=0, inplace=True)

**Promo2SinceYear**

In [ ]:
Store_df['Promo2SinceYear'].fillna(value=0, inplace=True)

**PromoInterval**

In [ ]:
Store_df['PromoInterval'].fillna(value=0, inplace=True)

In [ ]:
Store_df.isnull().sum()

**Combining The Two Datasets for ease of Analysis**

In [ ]:
Combined_Df=pd.merge(Rossmann_df, Store_df, on='Store' ,how='left')

In [ ]:
pd.DataFrame(Combined_Df)

**Changing Data Type**

**From Object to int64**

In StateHolidays

In [ ]:
Combined_Df['StateHoliday'].unique()

In [ ]:
Combined_Df['StateHoliday']=Combined_Df['StateHoliday'].replace(['0','a','b','c'],[0,1,2,3])
Combined_Df['StateHoliday'].astype('int64')
# Changed value and Datatype of StateHolidays

In [ ]:
Combined_Df['StateHoliday'].unique()

In Assortment

In [ ]:
Combined_Df['Assortment'].unique()

In [ ]:
Combined_Df['Assortment']=Combined_Df['Assortment'].replace(['a', 'c', 'b'],[0,1,2])
Combined_Df['Assortment'].astype('int64')
# Changed value and Datatype of Assortment

In [ ]:
Combined_Df['Assortment'].unique()

In StoreType

In [ ]:
Combined_Df['StoreType'].unique()

In [ ]:
Combined_Df['StoreType']=Combined_Df['StoreType'].replace(['c', 'a', 'd', 'b'],[0,1,2,3])
Combined_Df['StoreType'].astype('int64')
# Changed value and Datatype of StoreType

In [ ]:
Combined_Df['StoreType'].unique()

**From Object to DateTime**

In [ ]:
Combined_Df.info()

In [ ]:
Combined_Df['Date'] = pd.to_datetime(Combined_Df['Date'], format= '%Y-%m-%d')

In [ ]:
Combined_Df['CompetitionOpenSinceYear']=Combined_Df['CompetitionOpenSinceYear'].astype("int64")

In [ ]:
Combined_Df['CompetitionOpenSinceMonth']=Combined_Df['CompetitionOpenSinceMonth'].astype('int64')

In [ ]:
Combined_Df['CompetitionDistance']=Combined_Df['CompetitionDistance'].astype('int64')

In [ ]:
Combined_Df['Promo2SinceWeek']=Combined_Df['Promo2SinceWeek'].astype('int64')

In [ ]:
Combined_Df['Promo2SinceYear']=Combined_Df['Promo2SinceYear'].astype('int64')

Checking The Datatypes of the columns

In [ ]:
Combined_Df.info()

In [ ]:
Combined_Df.isnull().sum()

#**Exploratory Data analysis**

In [ ]:
Combined_Df.describe()

In [ ]:
Combined_Df.head()

**SALES**

In [ ]:
plt.figure(figsize=(15,7))
sns.pointplot(data=Combined_Df ,y='Sales', x='CompetitionOpenSinceYear')
plt.title('Sales vs Compititon open since year')

In [ ]:
plt.figure(figsize=(15,7))
sns.pointplot(data=Combined_Df ,y='Sales', x='CompetitionOpenSinceMonth')
plt.title('Sales vs Compititon open since Month')

In [ ]:
plt.figure(figsize=(15,6))
sns.pointplot(x= 'Promo2SinceYear', y= 'Sales', data=Combined_Df)
plt.title('Plot between Sales and Promo2 Since year')

In [ ]:
plt.figure(figsize=(20,7))
sns.pointplot(data=Combined_Df ,y='Sales', x='Customers')
plt.title('Sales vs Customers')